In [1]:
from etl import etl_bases_federais

In [2]:
etl_bases_federais()

https://portaldatransparencia.gov.br/download-de-dados/seguro-defeso SeguroDefeso
download/202403_SeguroDefeso.zip
202403_SeguroDefeso.csv
